<h1>Web Scraping using Beautiful Soup</h1>

Here, we first import the required packages

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

We then use the requests library to get the HTML document from the given Toronto Neighbourhood's Wikipedia page. We then use the <b>'lxml'</b> parser from BeautifulSoup to parse through the HTML code. 

In [4]:
r=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup=BeautifulSoup(r.text,'lxml')

We navigate to the div which contains the required table.

In [6]:
s1=soup.find("div",id="content").find("div",id="bodyContent").find("div",id="mw-content-text").find("div",class_="mw-parser-output").find("table")

We Parse through the Table and create a 2D list of all the Postalcodes, Boroughs and Neighbourhoods.

In [7]:
l=[]
for i in s1.find_all('tr'):
    if(i.th):
        l1=i.find_all('th')
        l1[0]=l1[0].text
        l1[1]=l1[1].text
        l1[2]=l1[2].text.strip('\n')
        l.append(l1)
    if(i.td !=None):
        l1=i.find_all('td')
        l1[0]=l1[0].text
        l1[1]=l1[1].text
        l1[2]=l1[2].text.strip('\n')
        l.append(l1)

Using this list, we create our primary data frame, which will be used for further refinement.

In [8]:
df=pd.DataFrame(l[1:],columns=l[0])

We drop those rows which have unassigned Boroughs.

In [9]:
df=pd.DataFrame(l[1:],columns=l[0])
x=(df['Borough']!='Not assigned')
df=df[x]

We reset the index, and then proceed to clean the data. When multiple Neighbourhoods exist for the same PostCode, we group them together. Also, if a neighbourhood  is not assigned, then we simply make the Borough it's neighbourhood

In [10]:
df=df.reset_index()
df=df.drop('index',axis=1)

y=set(df['Postcode'])
lomo=[]
moon=[]
y=list(y)
for i in y:
    x=df['Postcode']==i
    dftemp=df[x]
    l1=[]
    for j in dftemp['Neighbourhood']:
        if(j=='Not assigned'):
            yolo=dftemp['Borough']
            l1.append(yolo.iloc[0])
            continue
        l1.append(j)
    lomo.append(','.join(l1).strip(','))
    moon.append(dftemp['Borough'].iloc[0])

The final processed DataFrame is got created from the 3 lists.

In [13]:
df_final=pd.DataFrame({'Postcode':y,'Borough':moon,'Neighbourhood':lomo})
df_final.head()

,Postcode,Borough,Neighbourhood
0,M9W,Etobicoke,Northwest
1,M5S,Downtown Toronto,"Harbord,University of Toronto"
2,M3J,North York,"Northwood Park,York University"
3,M2H,North York,Hillcrest Village
4,M9C,Etobicoke,"Bloordale Gardens,Eringate,Markland Wood,Old B..."


In [14]:
df_final.shape

(103, 3)